In [1]:
import duckdb
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import requests

In [2]:
## General Bikeshare Feed Specification Standard
# gbfs = requests.get('https://gbfs.mex.lyftbikes.com/gbfs/gbfs.json').json()
gbfs = requests.get('https://gbfs.mex.lyftbikes.com/gbfs/en/station_information.json').json()

## Get the Stations Information
# stations_url =  gbfs['data']['es']['feeds'][0]['url']
stations_list =  gbfs['data']['stations']

In [3]:
listOfStations = pd.DataFrame(stations_list)

## List Of Stations SImplified
stations_simplified = listOfStations[['short_name','lat','lon']]

## List Of Stations Names
stations_names = listOfStations[['short_name','name']]


In [13]:
### Table of Stations
with duckdb.connect('/Users/gerardomartinez/Documents/ecobici_project/ecobici/backend/ecobici_stations.duckdb') as con:
    con.execute(f"""DROP TABLE IF EXISTS stations""")
    # Create the table
    con.execute("CREATE TABLE stations AS SELECT * FROM listOfStations")
    # Query the table
    result = con.execute("SELECT * FROM stations").fetchdf()

In [4]:
fileFolder = '/Volumes/T7 Shield/Ecobici/CLEAN'

rides_all = duckdb.sql(f""" 
SELECT * EXCLUDE(fecha_retiro,fecha_arribo),fecha_retiro_completa::date AS fecha_retiro,fecha_arribo_completa::date AS fecha_arribo   
    FROM read_parquet(['{fileFolder}/2025/*.parquet',
    '{fileFolder}/2024/*.parquet',
    '{fileFolder}/2023/*.parquet',
    '{fileFolder}/2022/*.parquet',
    '{fileFolder}/2021/*.parquet',
    '{fileFolder}/2020/*.parquet',
    '{fileFolder}/2019/*.parquet',
    '{fileFolder}/2018/*.parquet',
    '{fileFolder}/2017/*.parquet']) 
""")

rides_24 = duckdb.sql(f""" 
SELECT * EXCLUDE(fecha_retiro,fecha_arribo),fecha_retiro_completa::date AS fecha_retiro,fecha_arribo_completa::date AS fecha_arribo   
    FROM read_parquet([
    '{fileFolder}/2025/*.parquet',
    '{fileFolder}/2024/*.parquet'
]) 
""")

In [5]:
duckdb.sql("""
SELECT * FROM rides_24
""")

┌────────────────┬──────────────┬─────────┬───────────────────────┬─────────────────────┬──────────────────────┬─────────────────────┬───────────────────────┬───────────────────────┬────────────────────┬─────────────────┬────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────────┬──────────────┐
│ genero_usuario │ edad_usuario │  bici   │ ciclo_estacion_retiro │     hora_retiro     │ ciclo_estacionarribo │     hora_arribo     │ fecha_retiro_completa │ fecha_arribo_completa │ time_between_trips │ day_of_the_week │ is_weekend │     lat_start      │     lon_start      │      lat_end       │      lon_end       │  distance_meters   │ fecha_retiro │ fecha_arribo │
│    varchar     │    int64     │ varchar │        varchar        │ time with time zone │       varchar        │ time with time zone │       timestamp       │       timestamp       │       double       │     varchar     │   int32    │       double       │     